In [2]:
import itertools
import os
import sys
import glob
import time
import datetime
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns; sns.set(color_codes=True)
from pandas import DataFrame
from sklearn import mixture
from scipy import linalg
from sklearn.cluster import KMeans
from sklearn import datasets
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN

try:
    import sqlite3
except ImportError:
    print ('you need sqlite3 installed to use this program')
    sys.exit(0)
    
%matplotlib inline

In [3]:
def encode_string(s):
    """
    Simple utility function to make sure a string is proper
    to be used in a SQLite query
    (different than posgtresql, no N to specify unicode)
    EXAMPLE:
      That's my boy! -> 'That''s my boy!'
    """
    return "'"+s.replace("'","''")+"'"

In [4]:
dbfile = 'track_metadata.db'

In [5]:
# connect to the SQLite database
conn = sqlite3.connect(dbfile)

# from that connection, get a cursor to do queries
c = conn.cursor()

# so there is no confusion, the table name is 'songs'
TABLENAME = 'songs'

In [6]:
print ('*************** GENERAL SQLITE DEMO ***************************')

# list all tables in that dataset
# note that sqlite does the actual job when we call fetchall() or fetchone()
q = "SELECT name FROM sqlite_master WHERE type='table' ORDER BY name"
res = c.execute(q)
print ("* tables contained in that SQLite file/database (should be only 'songs'):")
print (res.fetchall())

*************** GENERAL SQLITE DEMO ***************************
* tables contained in that SQLite file/database (should be only 'songs'):
[('songs',)]


In [7]:
# list all columns names from table 'songs'
q = ("SELECT sql FROM sqlite_master WHERE tbl_name = 'songs' AND type = 'table'")
res = c.execute(q)
print ('* get info on columns names (original table creation command):')
print (res.fetchall()[0][0])

* get info on columns names (original table creation command):
CREATE TABLE songs (track_id text PRIMARY KEY, title text, song_id text, release text, artist_id text, artist_mbid text, artist_name text, duration real, artist_familiarity real, artist_hotttnesss real, year int, track_7digitalid int, shs_perf int, shs_work int)


In [8]:
q=("SELECT * FROM songs LIMIT 5")
res = c.execute(q)
print ('* get the whole table:')
print (res.fetchall())

* get the whole table:
[('TRMMMYQ128F932D901', 'Silent Night', 'SOQMMHC12AB0180CB8', 'Monster Ballads X-Mas', 'ARYZTJS1187B98C555', '357ff05d-848a-44cf-b608-cb34b5701ae5', 'Faster Pussy cat', 252.05506, 0.649822100201, 0.394031892714, 2003, 7032331, -1, 0), ('TRMMMKD128F425225D', 'Tanssi vaan', 'SOVFVAK12A8C1350D9', 'Karkuteillä', 'ARMVN3U1187FB3A1EB', '8d7ef530-a6fd-4f8f-b2e2-74aec765e0f9', 'Karkkiautomaatti', 156.55138, 0.439603966677, 0.356992107756, 1995, 1514808, -1, 0), ('TRMMMRX128F93187D9', 'No One Could Ever', 'SOGTUKN12AB017F4F1', 'Butter', 'ARGEKB01187FB50750', '3d403d44-36ce-465c-ad43-ae877e65adc4', 'Hudson Mohawke', 138.97098, 0.643680572058, 0.437503836595, 2006, 6945353, -1, 0), ('TRMMMCH128F425532C', 'Si Vos Querés', 'SOBNYVR12A8C13558C', 'De Culo', 'ARNWYLR1187B9B2F9C', '12be7648-7094-495f-90e6-df4189d68615', 'Yerba Brava', 145.05751, 0.448501159656, 0.372349068517, 2003, 2168257, -1, 0), ('TRMMMWA128F426B589', 'Tangle Of Aspens', 'SOHSBXH12A8C13B0DF', 'Rene Ablaze Pre

In [10]:
import csv
print ("Exporting data into CSV............")
cursor = conn.cursor()
cursor.execute("SELECT * FROM songs")
with open("full_metadata.csv", "w") as csv_file:
    csv_writer = csv.writer(csv_file, delimiter=",")
    csv_writer.writerow([i[0] for i in cursor.description])
    csv_writer.writerows(cursor)

Exporting data into CSV............


In [11]:
import pandas as pd

                                                          0
0         b80344d063b5ccb3212f76538f3d9e43d87dca9e\tSOAK...
1         b80344d063b5ccb3212f76538f3d9e43d87dca9e\tSOAP...
2         b80344d063b5ccb3212f76538f3d9e43d87dca9e\tSOBB...
3         b80344d063b5ccb3212f76538f3d9e43d87dca9e\tSOBF...
4         b80344d063b5ccb3212f76538f3d9e43d87dca9e\tSOBF...
...                                                     ...
48373581  b7815dbb206eb2831ce0fe040d0aa537e2e800f7\tSOUH...
48373582  b7815dbb206eb2831ce0fe040d0aa537e2e800f7\tSOUJ...
48373583  b7815dbb206eb2831ce0fe040d0aa537e2e800f7\tSOUS...
48373584  b7815dbb206eb2831ce0fe040d0aa537e2e800f7\tSOWY...
48373585  b7815dbb206eb2831ce0fe040d0aa537e2e800f7\tSOYY...

[48373586 rows x 1 columns]
